# Ingress

- [What is Ingress?](https://kubernetes.io/docs/concepts/services-networking/ingress/#what-is-ingress)
- [Must be enabled](IngressStart.ipynb)
- [Service Networking Concept](https://kubernetes.io/docs/concepts/services-networking/ingress/)
- Implemented by an [Ingress Controller](https://kubernetes.io/docs/concepts/services-networking/ingress-controllers/)
- Implementation dependent [Overview by Yitaek Hwang](https://medium.com/swlh/kubernetes-ingress-controller-overview-81abbaca19ec)
- Can include [External Oauth 2 ](https://kubernetes.github.io/ingress-nginx/examples/auth/oauth-external-auth/)
- Most popular [NGinx Ingress Controller](https://docs.nginx.com/nginx-ingress-controller/)

In [21]:
kubectl create deployment web --image=gcr.io/google-samples/hello-app:1.0

deployment.apps/web created


In [22]:
kubectl expose deployment web --type=NodePort --port=8080

service/web exposed


In [23]:
kubectl get service web 

NAME   TYPE       CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
web    NodePort   10.102.88.18   <none>        8080:30247/TCP   1s


In [24]:
NODEPORT=$(kubectl get service web  -o jsonpath='{.spec.ports[0].nodePort}')
echo $NODEPORT

30247


In [25]:
kubectl get pods

NAME                                    READY   STATUS    RESTARTS   AGE
kong-1618946942-kong-57d84768dc-76tzd   2/2     Running   0          3m48s
web-79d88c97d6-b784p                    1/1     Running   0          5s


In [26]:
echo $(minikube ip):$NODEPORT

192.168.49.2:30247


In [97]:
PROXY=$(kubectl get svc | awk '/kong-proxy/{print $1}')
echo $PROXY

kong-1618946942-kong-proxy


In [105]:
IPADDRESS=$(kubectl get svc $PROXY -o jsonpath='{.spec.clusterIP}')
echo $IPADDRESS

10.103.176.252


In [106]:
curl $IPADDRESS

{"message":"no Route matched with those values"}

In [108]:
DOMAIN=$IPADDRESS.nip.io
nslookup $DOMAIN

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
Name:	10.103.176.252.nip.io
Address: 10.103.176.252



In [64]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
spec:
  ingressClassName: kong
  rules:
    - host: $DOMAIN.nip.io
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: web
                port:
                  number: 8080
EOF

ingress.networking.k8s.io/example-ingress configured


In [65]:
kubectl get ingress example-ingress -o yaml

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"networking.k8s.io/v1","kind":"Ingress","metadata":{"annotations":{},"name":"example-ingress","namespace":"default"},"spec":{"ingressClassName":"kong","rules":[{"host":"10.103.176.252.nip.io","http":{"paths":[{"backend":{"service":{"name":"web","port":{"number":8080}}},"path":"/","pathType":"Prefix"}]}}]}}
  creationTimestamp: "2021-04-20T19:33:03Z"
  generation: 6
  name: example-ingress
  namespace: default
  resourceVersion: "2094"
  uid: 919ee2e2-707e-4025-aa03-9a2e9d7ccd18
spec:
  ingressClassName: kong
  rules:
  - host: 10.103.176.252.nip.io
    http:
      paths:
      - backend:
          service:
            name: web
            port:
              number: 8080
        path: /
        pathType: Prefix
status:
  loadBalancer:
    ingress:
    - ip: 10.103.176.252


In [66]:
kubectl describe ingress example-ingress

Name:             example-ingress
Namespace:        default
Address:          10.103.176.252
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host                   Path  Backends
  ----                   ----  --------
  10.103.176.252.nip.io  
                         /   web:8080 (172.17.0.4:8080)
Annotations:             <none>
Events:                  <none>


In [67]:
INGRESS_HOST=$(kubectl get ingress example-ingress -o jsonpath='{.spec.rules[0].host}')
echo $INGRESS_HOST

10.103.176.252.nip.io


In [68]:
curl $INGRESS_HOST/helloweb

Hello, world!
Version: 1.0.0
Hostname: web-79d88c97d6-b784p


## Create **more** services
We can say hello world in different ways

In [69]:
kubectl create deployment web2 --image=gcr.io/google-samples/hello-app:2.0


error: failed to create deployment: deployments.apps "web2" already exists


: 1

In [70]:
kubectl expose deployment web2 --port=8080 --type=NodePort


Error from server (AlreadyExists): services "web2" already exists


: 1

In [71]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
spec:
  ingressClassName: kong
  rules:
    - host: $DOMAIN.nip.io
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: web
                port:
                  number: 8080
          - path: /v2
            pathType: Prefix
            backend:
              service:
                name: web2
                port:
                 number: 8080
EOF

ingress.networking.k8s.io/example-ingress configured


In [73]:
curl $INGRESS_HOST/v2

Hello, world!
Version: 2.0.0
Hostname: web2-5d47994f45-s8jfb


In [74]:
kubectl get all

NAME                                        READY   STATUS    RESTARTS   AGE
pod/kong-1618946942-kong-57d84768dc-76tzd   2/2     Running   0          11m
pod/web-79d88c97d6-b784p                    1/1     Running   0          7m25s
pod/web2-5d47994f45-s8jfb                   1/1     Running   0          2m30s

NAME                                 TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)                      AGE
service/kong-1618946942-kong-proxy   LoadBalancer   10.103.176.252   10.103.176.252   80:31739/TCP,443:32408/TCP   11m
service/kubernetes                   ClusterIP      10.96.0.1        <none>           443/TCP                      35m
service/web                          NodePort       10.102.88.18     <none>           8080:30247/TCP               7m24s
service/web2                         NodePort       10.106.182.162   <none>           8080:31235/TCP               2m29s

NAME                                   READY   UP-TO-DATE   AVAILABLE   AGE
deployment.

## TLS

[ ![TLS](tls-ssl-handshake.png) ](https://www.cloudflare.com/learning/ssl/transport-layer-security-tls/)
- [OpenSSL](https://www.openssl.org/), command line
(https://wiki.openssl.org/index.php/Command_Line_Utilities)

In [75]:
mkdir -p openssl
cd openssl


/home/thomas/import/notebooks/openssl


In [76]:
DOMAIN=$INGRESS_HOST
CONF=${DOMAIN}.conf
SECRET=${DOMAIN}.yaml
KEY=${DOMAIN}.key
CRT=${DOMAIN}.crt

echo $DOMAIN

10.103.176.252.nip.io


In [77]:
pwd

/home/thomas/import/notebooks/openssl


In [78]:
ls

In [79]:
test  ! -f ${CONF}  && cat > ${CONF}  <<EOF
[ req ]
default_bits       = 2048
default_md         = sha512
default_keyfile    = ${KEYFILE}
prompt             = no
encrypt_key        = no
distinguished_name = req_distinguished_name
# distinguished_name
[ req_distinguished_name ]
countryName            = DE
localityName           = Berlin
organizationName       = nip.io
organizationalUnitName = K8S Demo Department
commonName             = *.${DOMAIN}
emailAddress           = nowhere@${DOMAIN}


EOF

In [80]:
echo $KEY $CONF $CRT

test  ! -f ${KEY} && openssl req -config ${CONF} -newkey rsa:2048 -nodes -keyout ${KEY} -x509 -out ${CRT}

10.103.176.252.nip.io.key 10.103.176.252.nip.io.conf 10.103.176.252.nip.io.crt
Generating a RSA private key
.............................................................+++++
.............................................................+++++
writing new private key to '10.103.176.252.nip.io.key'
-----


In [81]:
test ! -f ${SECRET} && cat > ${SECRET}<<EOF
apiVersion: v1
kind: Secret
metadata:
 name: wildcard.tls
 namespace: default
type: kubernetes.io/tls
data:
 tls.crt: `cat ${CRT} | base64 -w0`
 tls.key: `cat ${KEY} | base64 -w0`
EOF


In [82]:
cat $SECRET

apiVersion: v1
kind: Secret
metadata:
 name: wildcard.tls
 namespace: default
type: kubernetes.io/tls
data:
 tls.crt: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR3ekNDQXFzQ0ZHL2xnazMwNEkyQzNJczlUTVpOS2dab0JuL3NNQTBHQ1NxR1NJYjNEUUVCRFFVQU1JR2QKTVFzd0NRWURWUVFHRXdKRVJURVBNQTBHQTFVRUJ3d0dRbVZ5YkdsdU1ROHdEUVlEVlFRS0RBWnVhWEF1YVc4eApIREFhQmdOVkJBc01FMHM0VXlCRVpXMXZJRVJsY0dGeWRHMWxiblF4SURBZUJnTlZCQU1NRnlvdU1UQXVNVEF6CkxqRTNOaTR5TlRJdWJtbHdMbWx2TVN3d0tnWUpLb1pJaHZjTkFRa0JGaDF1YjNkb1pYSmxRREV3TGpFd015NHgKTnpZdU1qVXlMbTVwY0M1cGJ6QWVGdzB5TVRBME1qQXhPVFF3TXpoYUZ3MHlNVEExTWpBeE9UUXdNemhhTUlHZApNUXN3Q1FZRFZRUUdFd0pFUlRFUE1BMEdBMVVFQnd3R1FtVnliR2x1TVE4d0RRWURWUVFLREFadWFYQXVhVzh4CkhEQWFCZ05WQkFzTUUwczRVeUJFWlcxdklFUmxjR0Z5ZEcxbGJuUXhJREFlQmdOVkJBTU1GeW91TVRBdU1UQXoKTGpFM05pNHlOVEl1Ym1sd0xtbHZNU3d3S2dZSktvWklodmNOQVFrQkZoMXViM2RvWlhKbFFERXdMakV3TXk0eApOell1TWpVeUxtNXBjQzVwYnpDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTHRUCm5aWE10Z0xuckRiL0NISmI2OHF0V1pZUTM3K21RRmdRQS9IZnR5cU9wV1JRODhPZU

In [83]:
kubectl apply -f $SECRET

secret/wildcard.tls created


In [84]:
cd ..

In [85]:
pwd

/home/thomas/import/notebooks


In [86]:
kubectl delete ingress example-ingress

ingress.networking.k8s.io "example-ingress" deleted


In [87]:
kubectl get ingress

No resources found in default namespace.


In [88]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: website
spec:
  ingressClassName: kong
  tls:
    - hosts:
      - demo.$DOMAIN
      - info.$DOMAIN
      secretName: wildcard.tls
  rules:
  - host: info.$DOMAIN
    http:
      paths:
      - path: /web
        pathType: Prefix
        backend:
          service:
            name: web
            port:
              number: 8080
      - path: /web/v2
        pathType: Prefix
        backend:
          service:
            name: web2
            port:
              number: 8080
  - host: demo.$DOMAIN
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: web2
            port:
              number: 8080
EOF

ingress.networking.k8s.io/website created


In [109]:
curl -k https://info.$DOMAIN/web
curl -k https://info.$DOMAIN/web/v2
curl -k https://demo.$DOMAIN/

Hello, world!
Version: 1.0.0
Hostname: web-79d88c97d6-b784p
Hello, world!
Version: 2.0.0
Hostname: web2-5d47994f45-s8jfb
Hello, world!
Version: 2.0.0
Hostname: web2-5d47994f45-s8jfb


In [90]:
kubectl get ingress

NAME      CLASS   HOSTS                                                   ADDRESS   PORTS     AGE
website   kong    info.10.103.176.252.nip.io,demo.10.103.176.252.nip.io             80, 443   10s


In [ ]:
kubectl delete ingress website